**Drake Lyrics Scraping**
======
***

### Import Modules

In [10]:
import urllib.request as urllib2
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from bs4 import BeautifulSoup

## Part 1: Scrape Lyrics
***

### Read csv with song names

#### * <font color='gray'>I created a list of songs and albums I wanted to analyze (copied from metrolyrics)</font> *

In [11]:
quote_page = 'http://metrolyrics.com/{}-lyrics-drake.html'
filename = '/Users/sowmyamurali/Documents/DataViz/drakedata.csv'
songs = pd.read_csv(filename,encoding = "latin1")
songs = songs.replace("'", '',regex=True)
songs = songs.replace("&", '',regex=True)
songs = songs.replace("/", '',regex=True)

songs.album = songs.album.str.strip()
songs.song = songs.song.str.strip()

songs = songs.replace(' ', '-',regex=True)

### Scrape lyrics from Metrolyrics

In [12]:
#scrape metrolyrics for the lyrics
for index, row in songs.iterrows():
    page = urllib2.urlopen(quote_page.format(row['song']))
    soup = BeautifulSoup(page, 'html.parser')
    verses = soup.find_all('p', attrs={'class': 'verse'})

    lyrics = ''

    for verse in verses:
        text = verse.text.strip()
        text = re.sub(r"\[.*\]\n", "", unidecode(text))
        if lyrics == '':
            lyrics = lyrics + text.replace('\n', ' ')
        else:
            lyrics = lyrics + ' ' + text.replace('\n', ' ')

    songs.at[index, 'lyrics'] = lyrics

### Replace blanks with NAs, delete blank lyrics and clean up formating

In [13]:
songs = songs.replace('',np.nan, regex=True)
songs = songs.replace(',','', regex=True)
songs = songs.dropna(subset=['lyrics'], thresh=1)
songs['album'] = songs['album'].astype(str)
songs['album'] = songs['album'].str.lower()
songs = songs.replace('-', ' ',regex=True)

### Save scraped lyrics to csv

In [14]:
songs.to_csv('/Users/sowmyamurali/Documents/DataViz/drakedata_NEW.csv',index=False)